In [2]:
!pip install transformers datasets soundfile librosa


In [10]:

from transformers import WhisperForConditionalGeneration, WhisperProcessor, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Audio
from huggingface_hub import notebook_login
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

notebook_login()

In [4]:

# Disable wandb
import os
os.environ["WANDB_DISABLED"] = "true"

In [5]:
# Load model and processor
model_name = "farahabdou/whisper-arabic-english"
processor = WhisperProcessor.from_pretrained(model_name, language="en", task="translate")
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Load dataset with audio resampling
dataset = load_dataset("farahabdou/FLEURS-AR-EN-split", trust_remote_code=True)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/186M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2228 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/278 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/279 [00:00<?, ? examples/s]

In [6]:
# Preprocessing function
def prepare_dataset(batch):
    # Load audio
    audio = batch["audio"]
    
    # Process audio inputs
    input_features = processor.feature_extractor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"],
        return_tensors="pt"
    ).input_features[0]
    
    # Process text targets
    batch["labels"] = processor.tokenizer(
        text_target=batch["english"]
    ).input_ids
    
    batch["input_features"] = input_features
    return batch

# Apply preprocessing
tokenized_dataset = dataset.map(
    prepare_dataset,
    remove_columns=dataset["train"].column_names,
    num_proc=2
)

Map (num_proc=2):   0%|          | 0/2228 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/278 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/279 [00:00<?, ? examples/s]

In [7]:

# Data collator for padding
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [69]:
# -*- coding: utf-8 -*-


training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-ar-en-e2e",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    num_train_epochs=3,
    fp16=True,
    eval_strategy="epoch",  # Changed from evaluation_strategy
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_strategy="epoch",
    push_to_hub=True,
    hub_model_id="farahabdou/whisper-arabic-english-end2end",
    metric_for_best_model="comet",  # Or choose another metric
    load_best_model_at_end=True,
    report_to=None  # Explicitly disables all loggers
)





Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [67]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=processor.tokenizer,
    compute_metrics=compute_metrics
# Use the actual tokenizer (not feature extractor)
)

NameError: name 'compute_metrics' is not defined

In [12]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,2.060514
2,No log,1.835056


TrainOutput(global_step=417, training_loss=1.6082760744529376, metrics={'train_runtime': 2410.5415, 'train_samples_per_second': 2.773, 'train_steps_per_second': 0.173, 'total_flos': 1.91851574132736e+18, 'train_loss': 1.6082760744529376, 'epoch': 2.982078853046595})

In [32]:
!pip install -qU torch==2.0.1 transformers datasets soundfile librosa sacrebleu unbabel-comet==2.0.0

In [47]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)  # Move model to GPU/CPU

In [13]:
trainer.push_to_hub()

events.out.tfevents.1738057790.bd73b449a112.31.0:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/farahabdou/whisper-arabic-english-end2end/commit/3ee65898726c6c9e6ad3fd487a094d0152f89c03', commit_message='End of training', commit_description='', oid='3ee65898726c6c9e6ad3fd487a094d0152f89c03', pr_url=None, repo_url=RepoUrl('https://huggingface.co/farahabdou/whisper-arabic-english-end2end', endpoint='https://huggingface.co', repo_type='model', repo_id='farahabdou/whisper-arabic-english-end2end'), pr_revision=None, pr_num=None)